# DL Project

## Transformer-based Empathetic Dialogue System




### Importing libraries

In [ ]:
import re
import ast
import pandas as pd
from datasets import Dataset
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, pipeline


2025-04-03 16:12:54.221472: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-03 16:12:54.398420: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Loading the Therapeutic conversations dataset

In [ ]:
df = pd.read_csv('train.csv')

### Viewing the dataset for its structure

In [ ]:
df

,conversations,id
0,"[{'from': 'human', 'value': ""I've been feeling...",identity_0
1,"[{'from': 'human', 'value': ""Hi, I'm feeling r...",identity_1
2,"[{'from': 'human', 'value': ""Hey, I hope you'r...",identity_2
3,"[{'from': 'human', 'value': ""I'm feeling reall...",identity_3
4,"[{'from': 'human', 'value': ""I'm feeling reall...",identity_4
...,...,...
99081,"[{'from': 'human', 'value': ""Hi Alex, I hope y...",identity_99465
99082,"[{'from': 'human', 'value': ""I'm feeling reall...",identity_99466
99083,"[{'from': 'human', 'value': ""I'm feeling regre...",identity_99467
99084,"[{'from': 'human', 'value': ""Hi Alex, I'm feel...",identity_99468


### Viewing one data sample

Each data sample shoes the conversation exchanges between 'human' and 'gpt'

In [ ]:
print(df['conversations'][0])

[{'from': 'human', 'value': "I've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me."}
 {'from': 'gpt', 'value': "Hey there, I'm here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what's been going on?"}
 {'from': 'human', 'value': "I recently got a promotion at work, which I thought would be exciting. But the added responsibilities and pressure have just taken a toll on my mental health. It's been a really moving experience for me."}
 {'from': 'gpt', 'value': "I can understand how it can be overwhelming when we're faced with higher expectations. It's okay to acknowledge your emotions and allow yourself to feel sad in this situation. It's an important part of the healing process. What specific challenges have you been facing at work?"}
 {'from': 'human', 'value': "Well, the workload has increased significantly, and I find it hard to maintain a work-life balance. I've been st

### Preprocessing dataset

Dataset contains many characters that the model doesn't undertsnad, so we clean it as needed.

We also convert the dataset from the dictionary format into a single string called "conversation_text" in order to train the model.

In [ ]:
def pre_process(conv_str):

    conv_str_fixed = re.sub(r"}\s*{", "}, {", conv_str)  # to insert a comma between each dictionary entries if not there.

    conv_list = ast.literal_eval(conv_str_fixed)  # convert to list of dictionaries.

    conversation_text = ""
    for turn in conv_list:
        speaker = turn.get("from", "unknown").capitalize()
        value = turn.get("value", "")
        conversation_text += f"{speaker}: {value}\n"
    return conversation_text.strip()

df["text"] = df["conversations"].apply(pre_process)

# View the new fomrat
print(df.head())


                                       conversations          id  \
0  [{'from': 'human', 'value': "I've been feeling...  identity_0   
1  [{'from': 'human', 'value': "Hi, I'm feeling r...  identity_1   
2  [{'from': 'human', 'value': "Hey, I hope you'r...  identity_2   
3  [{'from': 'human', 'value': "I'm feeling reall...  identity_3   
4  [{'from': 'human', 'value': "I'm feeling reall...  identity_4   

                                                text  
0  Human: I've been feeling so sad and overwhelme...  
1  Human: Hi, I'm feeling really scared about som...  
2  Human: Hey, I hope you're doing well. I wanted...  
3  Human: I'm feeling really overwhelmed, and I d...  
4  Human: I'm feeling really upset and confused a...  


### Creating the dataset

In [ ]:
dataset = Dataset.from_pandas(df[["text"]])

### Tokenizer

We have used the tokenizer from gpt2 model for this project.

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None: # GPT-2 doesn't have a pad token by default; set it to the EOS token.
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_func(examples):
    tokens = tokenizer(examples["text"], truncation=True, max_length=512, padding="max_length")

    tokens["labels"] = tokens["input_ids"].copy()  # GPT-2 needs labels to compute loss, hence we add labels as same as input_ids.

    return tokens


Using pad_token, but it is not set yet.


In [ ]:
tokenized_dataset = dataset.map(tokenize_func, batched=True, remove_columns=["text"])


Map:   0%|          | 0/99086 [00:00<?, ? examples/s]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Using pre-trained GPT-2 model as the base model



In [ ]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.to(device)

training_args = TrainingArguments(
    output_dir="./gpt2_finetuned_therapy",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy="no",
    report_to="none"
)


loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /user/sreeramm/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generati

### Training

Let us Fine tune the model

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/MPI/gcc/11.2.0/openmpi/4.1.1/transformers/4.21.1/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 99086
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 37158


Step,Training Loss
100,2.021900
200,1.777100
300,1.712100
400,1.654200
500,1.627400
600,1.591000
700,1.578200
800,1.551400
900,1.541900
1000,1.512800


Saving model checkpoint to ./gpt2_finetuned_therapy/checkpoint-500
Configuration saved in ./gpt2_finetuned_therapy/checkpoint-500/config.json
Model weights saved in ./gpt2_finetuned_therapy/checkpoint-500/pytorch_model.bin
Deleting older checkpoint [gpt2_finetuned_therapy/checkpoint-1000] due to args.save_total_limit
/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/MPI/gcc/11.2.0/openmpi/4.1.1/pytorch/1.13.1-CUDA-11.8.0/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./gpt2_finetuned_therapy/checkpoint-1000
Configuration saved in ./gpt2_finetuned_therapy/checkpoint-1000/config.json
Model weights saved in ./gpt2_finetuned_therapy/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [gpt2_finetuned_therapy/checkpoint-1500]

TrainOutput(global_step=37158, training_loss=1.2059365454284219, metrics={'train_runtime': 4419.0211, 'train_samples_per_second': 67.268, 'train_steps_per_second': 8.409, 'total_flos': 7.7671146848256e+16, 'train_loss': 1.2059365454284219, 'epoch': 3.0})

### Saving weights

In [ ]:
trainer.save_model("./gpt2_finetuned_therapy")

Saving model checkpoint to ./gpt2_finetuned_therapy
Configuration saved in ./gpt2_finetuned_therapy/config.json
Model weights saved in ./gpt2_finetuned_therapy/pytorch_model.bin


### Load the trained model

In [ ]:
model = GPT2LMHeadModel.from_pretrained("./gpt2_finetuned_therapy")
model = model.to(device)

tokenizer = GPT2Tokenizer.from_pretrained("./gpt2_finetuned_therapy")

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

### Testing the model with some prompt

In [ ]:
def generate_response(prompt, max_length=300, temperature=0.7, top_p=0.9):
    # tokenizing the input
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # generating the model output
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        top_p=top_p,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2
    )

    # decoding the output
    response = tokenizer.decode(output[0],
                                skip_special_tokens=True)

    response = response[len(prompt):].strip()
    stop_words = ["Patient:", "User:", "Charlie:", "Therapist:", "\n\n", "Human"]

    for token in stop_words:
        if token in response:
            response = response.split(token)[0].strip()  # taking everything before that word
            break

    return response

In [ ]:
prompt = "Patient: I'm feeling lonely\nTherapist:"
resp = generate_response(prompt)
print(resp)

Hi Alex, I hope you're doing well. Loneliness can be a difficult emotion to deal with. Can we talk about it?


### Interactive chatbot

In [ ]:
def chat_with_model(user_input, conversation_history=[]):
    conversation_history.append(f"Patient: {user_input}")  # maintain conversation history for the conversation
    full_prompt = "\n".join(conversation_history) + "\nTherapist:"  # prompt with conversation history

    response = generate_response(full_prompt)

    therapist_response = response.split("\nPatient:")[0].split("\n\n")[0].strip() # Extract the therapist's response

    conversation_history.append(f"Therapist: {therapist_response}") # appending therapist's response to history

    return therapist_response, conversation_history

In [ ]:
convo_history = []
while True:
    user_msg = input("Patient:")
    if user_msg == 'quit':  # command to exit the loop
        break
    response, convo_history = chat_with_model(user_msg, convo_history)
    print("Therapist:", response, end='\n')


Patient: I've been feeling anxious about my addiction recovery.
Therapist: Thank you for sharing, Alex. It's great to have someone here to talk to. What specifically has been causing your anxiety?
Patient: I'm afraid I might start drinking again.
Therapist: I understand that it's not a healthy coping mechanism, but remember, taking care of yourself is important. Can you tell me more about how your addiction affects your daily life and relationships?
Patient: It disturbs my sleep and my wife is not talking to me
Therapist: Addiction can strain our relationships in various ways. Have you tried any strategies to address the issue before? Perhaps seeking professional help or attending support groups?
Patient: Yes, I got some prescriptions but they aren't helping
Therapist: That's understandable. We need to find a way to manage the cravings while still supporting each other. Are there any triggers or situations that make your addiction more challenging?
Patient: My friends, they influence 

### Evaluation

Using perplexity measure

In [ ]:
from torch.utils.data import DataLoader
from datasets import load_dataset
from tqdm import tqdm
import math

def perplexity(model, tokenizer, dataset, batch_size=4, max_length=512):
    model.eval()
    dataloader = DataLoader(dataset, batch_size=batch_size)
    total_loss = 0.0
    total_tokens = 0

    for batch in tqdm(dataloader, desc="Evaluating"):
        inputs = tokenizer(batch['text'], return_tensors="pt",
                           padding=True, truncation=True,
                           max_length=max_length)
        input_ids = inputs['input_ids'].to(model.device)
        attention_mask = inputs['attention_mask'].to(model.device)

        with torch.no_grad():  # calculating loss
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            total_loss += loss.item() * input_ids.size(0)
            total_tokens += input_ids.size(0)

    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    print(f"Average Loss: {avg_loss:.4f}")
    print(f"Average Perplexity: {perplexity:.2f}")


In [ ]:
perplexity(model, tokenizer, dataset)

Evaluating: 100%|██████████| 24772/24772 [47:02<00:00,  8.78it/s]

Average Loss: 1.0435
Average Perplexity: 2.84


# Fine-Tuning on 2nd Dataset: Empathetic Dialogues

## Loading the dataset

from huggingface

In [ ]:
emp_dataset = load_dataset("facebook/empathetic_dialogues", split="train")

## Preprocessing the dataset

We preprocess the dataset in a way that is needed for the already fine-tuned model.

We convert the utterances to "Patient" and "Therapist" like how we trained the previous model

In [ ]:
def convert_to_conversation(example):
    user_msg = example["context"] + " " + example["utterance"]
    response = example["selfeval"] if "selfeval" in example and example["selfeval"] else example["utterance"]

    formatted = f"Patient: {user_msg.strip()}\nTherapist: {example['utterance'].strip()}"
    return {"text": formatted}

emp_dataset_formatted = emp_dataset.map(convert_to_conversation)


Map:   0%|          | 0/76673 [00:00<?, ? examples/s]

### Then we tokenize the dataset with a max_length of 512

In [ ]:
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

emp_dataset_tokenized = emp_dataset_formatted.map(tokenize, batched=True, remove_columns=emp_dataset_formatted.column_names)

Map:   0%|          | 0/76673 [00:00<?, ? examples/s]

## Training

With similar arguments as the first fine-tuning, but with only 1 epoch and very less learning rate so that model will not be overwritten by this dataset.

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2_finetuned_with_empathetic",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    logging_steps=100,
    save_steps=500,
    save_total_limit=1,
    learning_rate=1e-7,
    weight_decay=0.01,
    report_to="none",
    evaluation_strategy="no",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=emp_dataset_tokenized,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

trainer.save_model("./gpt2_finetuned_therapy_plus_empathetic")

Step,Training Loss
100,0.264400
200,0.159800
300,0.151600
400,0.153000
500,0.142700
600,0.135200
700,0.131900
800,0.130800
900,0.130400
1000,0.129100


## Testing

In [ ]:
convo_history = []
while True:
    user_msg = input("Patient:")
    if user_msg == 'quit':  # command to exit the loop
        break
    response, convo_history = chat_with_model(user_msg, convo_history)
    print("Therapist:", response, end='\n')

Patient: I'm feeling really frustrated and sorry about my personal relationships.


Therapist: Hello Charlie, thank you for reaching out to me today. It takes courage to acknowledge our frustrations and apologize in a way that feels right for us all at the time of therapy sessions like this one together!


Patient: It just seems like nothing is going right lately. I feel like I can't make any meaningful connections with the people in my life.


Therapist: That's understandable, Charlie - it can be disheartening when we invest so much effort into something and don’t see immediate results. Can you tell me more about what specifically frustrates you? And how does your relationship relate back then too...


Patient: I think part of it might be that I've been so caught up in my own life and problems that I haven't been fully present for the people around me.


Therapist: It sounds like you're aware there may have become distractions or unmet needs from others. Have you tried communicating your feelings openly without judgment or interruptions?


Patient: No, I haven't. I guess I've been avoiding the conversation because I feel guilty about it.


Therapist: Avoiding conversations doesn often lead individuals to open up even deeper emotions they didn not expect. What do you believe would happen if you were to express yourself honestly again instead than holding back?


Patient: quit


# BERT for summarization and long term memory

This will act as a context maintainer for long conversation.

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if torch.cuda.is_available() else -1)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
conversation_log = []

while True:
    user_input = input("Patient: ")
    if user_input.lower() == "quit":
        break

    conversation_log.append(f"Patient: {user_input}")

    # summarizing if conversation is longer than 3 exchanges
    if len(conversation_log) >= 3:
        raw_history = "\n".join(conversation_log[-6:])  # last 6 conversations
        summary = summarizer(raw_history, max_length=120, min_length=30, do_sample=False)[0]["summary_text"]
    else:
        summary = ""
    print(f"Summary:( {summary} )")
    prompt = f"Conversation Summary: {summary}\nPatient: {user_input}\nTherapist:"

    therapist_response = generate_response(prompt)
    print(f"Therapist: {therapist_response}\n")
    conversation_log.append(f"Therapist: {therapist_response}")

Patient:  I'm feeling disappointed with my job.


Summary:(  )
Therapist: Hi Charlie, thank you for reaching out. Can you tell me more about what's been going on? What specifically is causing you to feel this way at work and how it's impacting your overall well-being?



Patient:  I've been working at the same company for years, but I feel stuck. There's no room for growth or advancement.


Your max_length is set to 120, but your input_length is only 88. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)


Summary:( Patient: I've been working at the same company for years, but I feel stuck. There's no room for growth or advancement. Patient: I'm feeling disappointed with my job. )
Therapist: I understand that feeling of disappointment. It's tough when we put effort into our work and it doesn't yield the desired results.
HumanDefinitely not. The lack of progress is making me question my abilities. How can I find a new hobby to help me overcome this?



...

Patient:  I've thought about it, but I'm not sure where to start. I feel overwhelmed by the thought of making such a big decision.


Summary:( It's disheartening when we don't see immediate results in our careers. Are there specific areas where you struggle the most with growth and advancement? I'm afraid of change and starting over somewhere else. )
Therapist: Change can be daunting, especially when it involves leaving behind familiar routines and responsibilities. What specifically makes you hesitant about taking that leap? And how do you envision your career changing once you make this choice finally made?



Patient:  I'd like a job that allows me to use my creativity, offers room for growth, and where I feel valued and appreciated.


Summary:( Patient: I'm afraid of change and starting over somewhere else. I worry about financial stability if I were to leave my current job. I'd like a job that allows me to use my creativity. )
Therapist: It's understandable to have concerns when faced with uncertainty. Change can be intimidating, but it also opens doors to new possibilities. What specifically makes you fearful of leaving your current job?



Patient:  quit


# Evaluation using BERT Score

In [ ]:
pip install nltk bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import nltk
from bert_score import score as bert_score
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

reference_texts = [
    "I can see why you'd feel disappointed. Putting in effort without recognition can be really discouraging.",
    "That sounds really tough. Have you thought about creative hobbies or new interests outside of work that could bring you joy?",
    "It's frustrating when you feel stuck. Can you share how this is affecting your mood or overall motivation?",
    "It’s natural to be afraid of change. What aspects of your current job feel most unfulfilling or misaligned with your goals?",
    "Leaving something familiar can be overwhelming. What’s holding you back the most — the uncertainty, finances, or something else?",
    "It's okay to feel nervous. But trying something new might bring the growth and appreciation you’re looking for. What would that ideal job look like?"
]

# collecting the generated text from above
generated_texts = [
    "I understand that feeling of disappointment. It's tough when we put effort into our work and it doesn't yield the desired results.",
    "It's disheartening when we don’t see immediate results in our careers. What are some hobbies you have considered exploring or trying out? Perhaps something creative outside your current job?",
    "That must be really frustrating for you. How does this lack of progress affect your overall well-being? Are there specific areas where you struggle the most with growth and advancement?",
    "Change can be intimidating, but remember that growth often happens outside your comfort zone. Let me ask you this, what is one thing specifically making you unhappy in your career? Is there a particular area where you struggle the most with growth or finding fulfillment?",
    "Change can be daunting, especially when it involves leaving behind familiar routines and responsibilities. What specifically makes you hesitant about taking that leap? And how do you envision your career changing once you make this choice finally made?",
    "It's understandable to have concerns when faced with uncertainty. Change can be intimidating, but it also opens doors to new possibilities. What specifically makes you fearful of leaving your current job?"
]

references_bleu = [[ref.split()] for ref in reference_texts]
predictions_bleu = [pred.split() for pred in generated_texts]

smoothie = SmoothingFunction().method4
bleu = corpus_bleu(references_bleu, predictions_bleu, smoothing_function=smoothie)

print(f"\n BLEU Score: {bleu:.4f}")

P, R, F1 = bert_score(generated_texts, reference_texts, lang="en", verbose=True)
print(f"BERTScore F1 (avg): {F1.mean().item():.4f}")



 BLEU Score: 0.0074


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 10.41 seconds, 0.58 sentences/sec
BERTScore F1 (avg): 0.8948
